In [ ]:
#!/usr/bin/env python
"""
This is a personal project to facilitate sending emails to multiple professors in the university's website.
This project follows the guidelines of Web Crawling and has not displayed any personal information of the professors
extracted and is used only for educational purposes.
This project is to be used only for educational and reference purposes and not to cause harm or any form of negative impact on anyone.

@Author = "Mamatha Juluru"
@Copyright = "Copyright 2021, The Email Automation Project"
@Date = August 10th, 2021
@Email = "mjuluru@asu.edu"
@Status = "Completed"
"""

In [9]:
!pip install beautifulsoup4 urllib3 plyer requests

In [184]:
# STEP1: Setup for web crawling

# Importing libraries for Crawling the webpages
import requests
from bs4 import BeautifulSoup

# Entering the url from which the professor's emails are extracted
url = "https://scai.engineering.asu.edu/faculty/"
# Entering headers - list browsers supportable
header = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}
# Getting the page contents
page = requests.get(url,headers=header)
# Create a crawler
soup = BeautifulSoup(page.content, 'html.parser')

In [160]:
# To decode the encoded emails extracted below
def decode_email(encoded_email):
    try:
        r = int(encoded_email[:2],16)
        email = ''.join([chr(int(encoded_email[i:i+2], 16) ^ r) for i in range(2, len(encoded_email), 2)])
        return email
    except (ValueError):
        pass

In [194]:
# STEP2: Perform web crawling and parse the webpages

# Crawling the page to extract the names and emails of a professor in SCAI department of ASU
table = soup.find("div", attrs={'class':'et_builder_inner_content et_pb_gutters3'})
content_containers = table.find_all("div",class_= "et_pb_ajax_pagination_container")
prof_details = []
prof_emails = []
for container in content_containers:
  articles = container.find_all("article")
  for article in articles:
    prof_name = article.find_all("h2")
    prof_name = prof_name[0].find("a").text
    prof_email = article.find_all("span", class_="__cf_email__")[0]['data-cfemail']
    prof_email = decode_email(prof_email)
    prof_details.append({
        'name':prof_name,
        'email':prof_email,
    })
    prof_emails.append(prof_email)
# Due to prviacy concerns cannot print the original details of the professors

In [185]:
# STEP3: Setup for automatic email sending to the professors

# Importing libraries required to send automated emails
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
import smtplib
import os

In [186]:
#Initializing connection to the email server
smtp = smtplib.SMTP('smtp.gmail.com',587)
smtp.ehlo()
smtp.starttls()
smtp.login('<from email>','<app password>')

(235, b'2.7.0 Accepted')

In [187]:
# STEP4: Code to create a message

#Creating the message
def message_draft(subject = "Python Webcrawler notification",
            text = "", attachments=None, images=None):
  #Building message
  message = MIMEMultipart()
  # Adding subject line to the email
  message['Subject'] = subject
  # Adding content of the email
  message.attach(MIMEText(text))
  # Adding images
  if images is not None:
    if type(images) is not list:
      images = [images]
    for image in images:
      # Read the binary data
      image_data = open(image,'rb').read()
      # Attach image to the message
      message.attach(MIMEImage(image_data,
                               name=os.path.basename(image)))
  # Adding attachments
  if attachments is not None:
    if type(attachments) is not list:
      attachments = [attachments]
    for attachment in attachments:
      # Open the attachment
      with open(attachment, 'rb') as f:
        # Read the attachment contents
        file = MIMEApplication(f.read(),
                               name=os.path.basename(attachment))
        file['Content-Disposition'] = f'attachemnt;\
        filename="{os.path.basename(attachment)}"'
        # Attach image to the message
        message.attach(file)
  return message


In [ ]:
# STEP5: Send the emails

subject = "Inquiry Regarding Available Grader and TA Positions"
body = """Dear Professor,

I hope this email finds you well. My name is Mamatha Juluru, and I am currently pursuing a Master\'s degree in Computer Science at Arizona State University. I have been following your work and research with great interest, and I am reaching out to inquire about any potential grader or TA positions available within your departments.

With a strong educational background and hands-on experience as a Software Developer and Data Scientist at UnitedHealth Group and Capgemini, I have honed my skills in Python, Java, SQL, Software Development, and Data Visualization. I have successfully contributed to projects involving software development, data governance, and complex data analysis.

The opportunity to work as a grader or TA under your guidance would be an invaluable experience for me, allowing me to share my practical knowledge with fellow students and contribute to the academic community. I am excited to collaborate, assist students, and contribute to the academic environment at your esteemed institution.

Please find my resume attached.

Thank you for considering my application. I would be grateful for the chance to discuss how my skills and experience align with your department\'s needs. Please feel free to contact me at mjuluru@asu.edu or +1 (xxx) xxx xxxx.


Warm Regards,
Mamatha Juluru.
+1 (xxx) xxx xxxx.
LinkedIn: https://www.linkedin.com/in/mamathaj/"""

# Creating message to send in email
msg = message_draft(subject, body, r"/content/drive/MyDrive/WebCrawler_Python/Mamatha_Juluru_Resume.pdf")

# prof_emails : List of Professor emails in the School of Augmented Intelligenceto = []+[]+ (SCAI) in Arizona State University (ASU) extracter earlier in the code
# Sending emails
smtp.sendmail(from_addr="<from email>", to_addrs=prof_emails, msg = msg.as_string())


In [ ]:
# STEP6: End the connection

# Closing the server connection
smtp.quit()